In [ ]:
from neuron import h
import os
import numpy as np
import matplotlib.pyplot as plt
from plots import plot_cell, plot_cell_with_electrode, plot_bc_rgc
from BC_model_template import ONBipolarCell

In [ ]:
# Load compiled channel mechanisms and standard run system
h.nrn_load_dll("/Users/lillikiessling/Documents/Stanford/Code/BC_model/Ion_channels/arm64/libnrnmech.dylib")
h.load_file('stdrun.hoc') 
#h.load_file("nrn/cells/gc.nrn")            # defines soma, dend1[], dend2[], dend3[], dend4[], dend5[], dend6[]
h.load_file("RGC_model_Anna/RGC_20171004_S1W3C3_ON_S_200um_dend_low_14um_diameter_3um_axon_up_6um_axon_sections_20_axon_points_3000_axon_diam_1um.hoc")  # defines soma, dend[], axon[]
h.load_file("nrn/cells/BC_Euler9_ON.hoc")

### 1. RGC Cell

In [ ]:
rgc_cell = h.RGC_20171004_S1W3C3_ON_S_200um_dend_low_14um_diameter_3um_axon_up_6um_axon_sections_20_axon_points_3000_axon_diam_1um()
for sec in rgc_cell.all:
    sec.g_pas = 1/70000.0

plot_cell(rgc_cell)

### 2. BC Cell

In [ ]:
bc_cell = ONBipolarCell()
# Choose one Terminal for voltage clamp
term_list = list(bc_cell.terminal)
preSyn = term_list[0]

for sec in bc_cell.all:
    sec.cao = 1.8 #mM
    sec.cai = 0.0001 #mM

preSyn.gcabar_Bip_CaL = 5 # TODO: verify hyperparameter

plot_cell(bc_cell)

### 3. Connect with ribbon tiger synapse

In [ ]:
plot_bc_rgc(bc_cell, rgc_cell)

def make_syn_at(sec, idx):
    """Create a ribbon_tiger at sec(0.5) with Ca pointers hooked up."""
    syn = h.ribbon_tiger_simple(sec(0.5))
    h.setpointer(preSyn(0.5).ca_ion._ref_cai,  'preCA', syn)
    return syn

#MAPPING TABLE
mapped_dend1 = [26,27,28,29,30,31,32,33,34,35,36,37,38,39]   # 14
mapped_dend2 = [62,63,64]                                   # 3
mapped_dend3 = [21]                                         # 1
mapped_dend4 = [59]                                         # 1
mapped_dend5 = [60]                                         # 1
mapped_dend6 = [0,1,2]                                      # 3

# Full list of 23 synaptic *locations*
syn_locs = (
    mapped_dend1 +
    mapped_dend2 +
    mapped_dend3 +
    mapped_dend4 +
    mapped_dend5 +
    mapped_dend6
)

assert len(syn_locs) == 23

# Allocate arrays
synapse  = [None]*23
synapse2 = [None]*23
synapse3 = [None]*23

# Create synapses for all 23 locations × 3 ribbons
for idx, dend_idx in enumerate(syn_locs):
    sec = rgc_cell.dend[dend_idx]
    synapse[idx]  = make_syn_at(sec, idx)
    synapse2[idx] = make_syn_at(sec, idx)
    synapse3[idx] = make_syn_at(sec, idx)

def set_stim(GMAX=0.00256*1.7, GMAXN=0.00011*1.7):
    for arr in (synapse, synapse2, synapse3):
        for s in arr:
            if s is None:
                continue
            s.gmax       = GMAX
            s.gmaxN      = GMAXN
            s.mg         = 0.5
set_stim()